In [ ]:
import os
import time
from dotenv import load_dotenv
import requests
import json

### 환경 변수 로드

load_dotenv()

### JSON 파일에서 토큰 로드

def load_token(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        return None
        
token_deviceId = load_token('token_deviceId.json')

ACCESS_TOKEN = token_deviceId.get('token')
DEVICE_ID = token_deviceId.get('device_id')

ACCESS_TOKEN, DEVICE_ID

### Heyhome Config 로드

BASE_URL = os.getenv('BASE_URL')

# 헤더 설정
HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json",
}

def control_power_strip_ports(device_id, states, description):
    """
    멀티탭 포트 상태를 제어하는 함수.
    :param device_id: 제어할 장치의 ID
    :param states: 포트 상태 딕셔너리
    :param description: 단계 설명
    """
    url = f"{BASE_URL}/control/{device_id}"
    payload = {"requirments": states}

    try:
        response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
        if response.status_code == 200:
            print(f"Description: {description}, {states}")
        else:
            print(f"Failed to update device {device_id}. Status: {response.status_code}, Response: {response.text}")
    except requests.RequestException as e:
        print(f"Error during API request: {e}")

def cycle_control(device_id):
    """
    주기적으로 멀티탭 포트 상태를 제어하는 함수.
    에러 발생 시 프로그램이 멈추지 않도록 예외 처리를 추가.
    :param device_id: 제어할 장치의 ID
    """
    steps = [
        {"description": "Turning ON fog", "states": {"power1": True, "power2": False, "power3": True}, "delay": 10},
        {"description": "Turning OFF fog", "states": {"power1": False, "power2": False, "power3": True}, "delay": 10},
        {"description": "Turning ON plasma", "states": {"power1": False, "power2": True, "power3": True}, "delay": 10},
        {"description": "Turning OFF plasma", "states": {"power1": False, "power2": False, "power3": True}, "delay": 10},
        {"description": "Turning ON fog and plasma", "states": {"power1": True, "power2": True, "power3": True}, "delay": 10},
        {"description": "Turning OFF fog and plasma", "states": {"power1": False, "power2": False, "power3": True}, "delay": 10},
    ]

    while True:
        for step in steps:
            try:
                control_power_strip_ports(device_id, step["states"], step["description"])
            except Exception as e:
                print(f"Error during step: {step['description']}. Error: {e}")
            finally:
                time.sleep(step["delay"])  # 각 단계별 대기 시간

def main():
    # DEVICE_ID = "50450710e8db84f198f8"  # 멀티탭 장치 ID

    # # 데이터베이스 초기화
    # initialize_db()

    try:
        print("Starting cycle control...")
        cycle_control(DEVICE_ID)
    except KeyboardInterrupt:
        print("Cycle control stopped manually.")

if __name__ == "__main__":
    main()